<a href="https://colab.research.google.com/github/FabioMMaia/LLMs/blob/main/Hello_World2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch

# Verifica se o PyTorch está usando GPU
if torch.cuda.is_available():
    # Obtém o objeto da GPU
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    print(gpu_info)
else:
    print("GPU não está disponível no Colab.")

Fri Jan 12 00:05:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0              31W /  70W |   9097MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.7 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 4.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 4.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 16.8 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 19.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.8 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 7.1 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-6y20wb9r/overlay/local/bin
    changing mode of /tmp/pip-build-env-6y20wb9r/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-6y20wb9r/overlay/lo

In [2]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [3]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [4]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [ ]:
n_gpu_layers = int(0.8 * 15360)

In [13]:
int(0.8 * 15360)

12288

In [14]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    # n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_batch = 8192,
    # n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    n_gpu_layers=64 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [15]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

64

In [16]:
prompt = "Write a linear regression in python"
prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: {prompt}

ASSISTANT:
'''

In [17]:
prompt_template

'SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.\n\nUSER: Write a linear regression in python\n\nASSISTANT:\n'

In [20]:
import time

# Defina a classe Timer para o contexto
class Timer:
    def __enter__(self):
        self.start_time = time.time()
        return self

    def __exit__(self, *args):
        self.end_time = time.time()
        elapsed_time = self.end_time - self.start_time
        print(f"Tempo de execução: {elapsed_time:.2f} segundos")

n_gpu_layers:64, n_batch:512


In [24]:
# response=lcpp_llm(prompt=prompt_template, max_tokens=512, temperature=0.5, top_p=0.95,
#                   repeat_penalty=1.2, top_k=150,
#                   echo=True)

with Timer():
  print(f'n_gpu_layers:{lcpp_llm.params.n_gpu_layers}, n_batch:{lcpp_llm.params.n_batch}')
  response = lcpp_llm(prompt=prompt_template,
                      max_tokens=512,
                      temperature=0.5,
                      top_p=0.95,
                      repeat_penalty=1.2,
                      top_k=150,
                      echo=True)



n_gpu_layers:64, n_batch:512


Llama.generate: prefix-match hit


Tempo de execução: 23.62 segundos


In [25]:
print(response)

{'id': 'cmpl-71e1e4f6-4e80-4499-95ab-c3dd9ba27e0b', 'object': 'text_completion', 'created': 1705018472, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': "SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.\n\nUSER: Write a linear regression in python\n\nASSISTANT:\nHello! I'd be happy to help you with that. A linear regression in Python can be implemented using the scikit-learn library. Here is an example of how you could do this:\n```\nfrom sklearn.linear_model import LinearRegression\nimport pandas as pd\n\n# Load your dataset into a Pandas DataFrame\ndf = pd.read_csv('your_data.csv')\n\n# Create a new feature to be used for the linear regression\nX = df[['feature1', 'feature2']]\ny = df['target']\n\n# Split the data into training and testing sets\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_siz

In [26]:
print(response["choices"][0]["text"])

SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: Write a linear regression in python

ASSISTANT:
Hello! I'd be happy to help you with that. A linear regression in Python can be implemented using the scikit-learn library. Here is an example of how you could do this:
```
from sklearn.linear_model import LinearRegression
import pandas as pd

# Load your dataset into a Pandas DataFrame
df = pd.read_csv('your_data.csv')

# Create a new feature to be used for the linear regression
X = df[['feature1', 'feature2']]
y = df['target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Create a linear regression object and fit it to the training data
reg = LinearRegression()
reg.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = reg.predict(X_test)

# Evaluate the performance of the model using mean squared error
mse = np.mean((y_test - y_pred) ** 2)
print('Me

# Teste 2

In [28]:
import os

# Obter o número de núcleos da CPU
num_cores = os.cpu_count()

print(f"Número de núcleos da CPU: {num_cores}")

Número de núcleos da CPU: 2


In [34]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    # n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_batch = 1028,
    # n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    n_gpu_layers=64 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [37]:
with Timer():
  print(f'n_gpu_layers:{lcpp_llm.params.n_gpu_layers}, n_batch:{lcpp_llm.params.n_batch}')
  response = lcpp_llm(prompt=prompt_template, max_tokens=1028, temperature=0.3, top_p=0.95,
                        repeat_penalty=1.2, top_k=150, echo=True)

n_gpu_layers:64, n_batch:512


Llama.generate: prefix-match hit


Tempo de execução: 16.76 segundos


In [38]:
print(response)

{'id': 'cmpl-92f2e322-03c2-4a59-9368-69f103c37c08', 'object': 'text_completion', 'created': 1705018933, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': "SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.\n\nUSER: Write a linear regression in python\n\nASSISTANT:\n\nTo write a linear regression in Python, you can use the scikit-learn library. Here's an example of how to do it:\n```\nfrom sklearn.linear_model import LinearRegression\nimport pandas as pd\n\n# Load your dataset into a Pandas DataFrame\ndf = pd.read_csv('your_data.csv')\n\n# Create a linear regression object and fit the data\nreg = LinearRegression()\nreg.fit(df[['x1', 'x2']], df['y'])\n\n# Print the coefficients\nprint(reg.coef_)\n\n# Print the R-squared value\nprint(reg.score(df[['x1', 'x2']], df['y']))\n```\nThis code will load your dataset into a P

In [39]:
print(response["choices"][0]["text"])

SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: Write a linear regression in python

ASSISTANT:

To write a linear regression in Python, you can use the scikit-learn library. Here's an example of how to do it:
```
from sklearn.linear_model import LinearRegression
import pandas as pd

# Load your dataset into a Pandas DataFrame
df = pd.read_csv('your_data.csv')

# Create a linear regression object and fit the data
reg = LinearRegression()
reg.fit(df[['x1', 'x2']], df['y'])

# Print the coefficients
print(reg.coef_)

# Print the R-squared value
print(reg.score(df[['x1', 'x2']], df['y']))
```
This code will load your dataset into a Pandas DataFrame, create a linear regression object and fit the data using the `fit()` method. It will then print the coefficients of the linear regression and the R-squared value.

Please note that you need to replace `'your_data.csv'` with the name of your own dataset file. Also, make sure that your dataset has a

In [43]:
prompt_aux = response["choices"][0]["text"]

prompt_aux += '''

USER: Can you say the same in portuguese?

ASSISTANT:
'''

print(prompt_aux)

SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: Write a linear regression in python

ASSISTANT:

To write a linear regression in Python, you can use the scikit-learn library. Here's an example of how to do it:
```
from sklearn.linear_model import LinearRegression
import pandas as pd

# Load your dataset into a Pandas DataFrame
df = pd.read_csv('your_data.csv')

# Create a linear regression object and fit the data
reg = LinearRegression()
reg.fit(df[['x1', 'x2']], df['y'])

# Print the coefficients
print(reg.coef_)

# Print the R-squared value
print(reg.score(df[['x1', 'x2']], df['y']))
```
This code will load your dataset into a Pandas DataFrame, create a linear regression object and fit the data using the `fit()` method. It will then print the coefficients of the linear regression and the R-squared value.

Please note that you need to replace `'your_data.csv'` with the name of your own dataset file. Also, make sure that your dataset has a

In [61]:
print(f'n_gpu_layers:{lcpp_llm.params.n_gpu_layers}, n_batch:{lcpp_llm.params.n_batch}')

n_gpu_layers:64, n_batch:512


In [71]:
with Timer():
  response = lcpp_llm(prompt=prompt_aux,
                      max_tokens=2048,
                      temperature=0.3,
                      top_p=0.95,
                      repeat_penalty=1.2,
                      top_k=150,
                      echo=True)

Llama.generate: prefix-match hit


Tempo de execução: 12.76 segundos


In [72]:
print(response["choices"][0]["text"])

SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: Write a linear regression in python

ASSISTANT:

To write a linear regression in Python, you can use the scikit-learn library. Here's an example of how to do it:
```
from sklearn.linear_model import LinearRegression
import pandas as pd

# Load your dataset into a Pandas DataFrame
df = pd.read_csv('your_data.csv')

# Create a linear regression object and fit the data
reg = LinearRegression()
reg.fit(df[['x1', 'x2']], df['y'])

# Print the coefficients
print(reg.coef_)

# Print the R-squared value
print(reg.score(df[['x1', 'x2']], df['y']))
```
This code will load your dataset into a Pandas DataFrame, create a linear regression object and fit the data using the `fit()` method. It will then print the coefficients of the linear regression and the R-squared value.

Please note that you need to replace `'your_data.csv'` with the name of your own dataset file. Also, make sure that your dataset has a

In [73]:
len(response["choices"][0]["text"])

1648

In [74]:
import json
# Salvar a resposta em um arquivo JSON
with open('output.json', 'w') as f:
    json.dump(response, f)

In [66]:
import os
os.getcwd()

'/content'